In [235]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
from pyproj import Proj, transform

InteractiveShell.ast_node_interactivity = "all"

Lat/Lon Box
- North: 47.8
- South: 47.4
- West: -122.5
- East: -122.2


- month
- day of week
- time of day


- Fatalities
- Weather conditions
- DUIs
- Cause of accident

In [236]:
df = pd.read_csv('~\Desktop\WA_Crash_Data.csv', sep=',')
df.shape

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (23,29,30,31,47,51,58,94,111,112,115,181,207,212) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(289671, 217)

In [237]:
df = df.drop(df[np.isnan(df.Colli_Dtl_Info_State_Plane_X)].index)
df = df.drop(df[np.isnan(df.Colli_Dtl_Info_State_Plane_Y)].index)
df.shape

(180145, 217)

Function to convert stateplane coordinates to latlons. Run, and subset on coordinate grid.

In [238]:
def convert_stateplane_to_latlon(state_x, state_y, proj_in=2286, proj_out=4326):
    inProj = Proj(init='epsg:' + str(proj_in), preserve_units=True)
    outProj = Proj(init='epsg:' + str(proj_out))
    x2, y2 = transform(inProj, outProj, state_x, state_y)
    return y2, x2

In [239]:
x, y = np.array(df.Colli_Dtl_Info_State_Plane_X), np.array(df.Colli_Dtl_Info_State_Plane_Y)
x_new, y_new = convert_stateplane_to_latlon(x, y)
df = df.rename(columns={'Colli_Dtl_Info_State_Plane_X':'lat',
                   'Colli_Dtl_Info_State_Plane_Y':'lon'})
df['lat'] = pd.DataFrame(x_new)
df['lon'] = pd.DataFrame(y_new)
df.shape

(180145, 217)

In [240]:
df = df.drop(df[df.lat > 47.8].index)
df = df.drop(df[df.lat < 47.4].index)
df = df.drop(df[df.lon > -122.2].index)
df = df.drop(df[df.lon < -122.5].index)
df.shape

(82959, 217)

Prints collision data to csv

In [195]:
#df[['lat', 'lon']].to_csv('WSP_Crash_Data.csv', sep=',')

Remove columns that have blanks for 50% or more of the observations

In [242]:
df_temp = df.loc[:, df.count(axis=0).values >= 0.8*df.shape[0]]
df_temp.shape

(82959, 55)

In [232]:
df_temp.to_csv('~\Desktop\WA_Crash_Data_out2.csv', sep=',')